In [1]:
# pdf distinction
# we have to distinguish the format of the pdf and then we need to choose the French one
# there are three different formats already detected
# 1) two column documents (one column for French one column for Dutch)
# The two column documents can be horizontal or vertical orientated.
# 1.a) there is a line between them
# 1.b) there is no line between them
# 2) one language after the other format
# 3) These texts start as two column for very first pages then continued like the other

In [2]:
from pdf2image import convert_from_path 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

In [ ]:
def language_detection(text):
    '''This function takes a string and detects its language with fasttext and retuns the language'''
    return (language, text_in_fr)


In [3]:
def Welcome(pdf_file):
    '''
    This function:
    -takes a pdf document path as an input, 
    -reads them as PIL.PpmImagePlugin.PpmImageFile
    -checks the orienatations of the pages and rotates accordingly
    -splits the pages if they are two column formatted
    -extracts the texts
    -devide the texts according the language
    -returns whole text in French
    '''
    # read the documents 
    pages = convert_from_path(pdf_file, 500) # store all the pages of pdf file
    whole_text = ""
    for page in pages:
        report = pytesseract.image_to_osd(page)
        
        if report.split("\n")[1].split(":")[1].replace(" ", "") == "270":
            img = np.array(page) # convert PIL.PpmImagePlugin.PpmImageFile to np.array
            img_pil = Image.fromarray(np.uint8(img)).convert('RGB') # np.array to PIL image
            rotated  = img_pil.transpose(Image.ROTATE_270) #rotate the image 
            img = np.array(rotated)# convert the page to np array to see as image
        
        elif report.split("\n")[1].split(":")[1].replace(" ", "") == "90":
            img = np.array(page) # convert PIL.PpmImagePlugin.PpmImageFile to np.array
            img_pil = Image.fromarray(np.uint8(img)).convert('RGB') # np.array to PIL image
            rotated  = img_pil.transpose(Image.ROTATE_90) #rotate the image 
            img = np.array(rotated)# convert the page to np array to see as image
        
        else:
            # convert the page to np array to see as image
            img = np.array(page) 
        # splitting the array into 51% sliced pieces vertically
        left_side = img[:,:-(round(img.shape[1]*0.49))]

        right_side = img[:,(round(img.shape[1]*0.49)):]

        # read the text for each half with pytessaract
        text_1 = str(((pytesseract.image_to_string(left_side, lang ="fra")))) 
        text_2 = str(((pytesseract.image_to_string(right_side, lang ="fra")))) 
        
        # if language detection of each half is equal, the page is not in two column format
        # that means we have one language follow the other or all in french text
        if language_detection(text_1) == language_detection(text_2):
            # assign the detected text from whole page as a text
            text = str(((pytesseract.image_to_string(img, lang ="fra"))))
            # take only the french part of the text
            if language_detection(text)[0] == "fr":
                fr_text = language_detection(text)[1]
        # otherwise the page is two column page. We need to find out the French half and assign the text    
        elif language_detection(text_1) == "fr":
            fr_text = text_1 
        else: 
            fr_text = text_2
        # ansamble all page texts a genearal text of the pdf document
        whole_text = whole_text + fr_text
    return whole_text